In [45]:
!pip install jupyternotify

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for jupyternotify: filename=jupyternotify-0.1.15-py3-none-any.whl size=8725 sha256=40cee503bacf55c3fb580275d6db8d84527f005d56926ca344e2ace87019c778
  Stored in directory: c:\users\ronaa\appdata\local\pip\cache\wheels\db\f4\43\06c94fe0f5bacf0029ea8ebb8d080f372b97661740be7b3d74
Successfully built jupyternotify


In [1]:
from kiteconnect import KiteConnect
from selenium import webdriver
import time
import os
import re

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
from matplotlib import pyplot as plt
import os
import sqlite3

In [52]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [3]:
apikey=""
apisecret=""

In [4]:
kite=KiteConnect(api_key=apikey)

In [5]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=kdwok1gk1epq3k3k&v=3


In [6]:
data=kite.generate_session("kZKrg13b8Az2egjNsrQRL6NdHofzKs1N",api_secret=apisecret)

In [7]:
data

{'user_type': 'individual',
 'email': 'the.ravina@gmail.com',
 'user_name': 'Ravina Vinayak More',
 'user_shortname': 'Ravina',
 'broker': 'ZERODHA',
 'exchanges': ['CDS', 'BSE', 'NSE', 'NFO', 'BFO', 'MF'],
 'products': ['CNC', 'NRML', 'MIS', 'BO', 'CO'],
 'order_types': ['MARKET', 'LIMIT', 'SL', 'SL-M'],
 'avatar_url': None,
 'user_id': 'LF9990',
 'api_key': 'kdwok1gk1epq3k3k',
 'access_token': '7nC76gxoX3xuhAxt6Rzw0hk27DBktT3j',
 'public_token': 'osrYpSuvpE740w1h6nFwyb3BcKYetktn',
 'refresh_token': '',
 'enctoken': 'wO4GQ0NIebyf2LQALBwrOu1SQm6ny1BZwxL3MhimWMKFkiYt63wAYNbsnasdoXrFNHkHNtvSiqYEv/hLOP1+oFUYfXINyTr/OTRNfd843rgXUfWZ2lX63PLNCP+XO2o=',
 'login_time': datetime.datetime(2023, 5, 13, 11, 27, 15),
 'meta': {'demat_consent': 'physical'}}

In [8]:
kite.set_access_token(data["access_token"])

In [9]:
kite.instruments("NSE")

[{'instrument_token': 264713,
  'exchange_token': '1034',
  'tradingsymbol': 'HANGSENG BEES-NAV',
  'name': 'HANGSENG BEES-NAV',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264969,
  'exchange_token': '1035',
  'tradingsymbol': 'INDIA VIX',
  'name': 'INDIA VIX',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 260617,
  'exchange_token': '1018',
  'tradingsymbol': 'NIFTY 100',
  'name': 'NIFTY 100',
  'last_price': 0.0,
  'expiry': '',
  'strike': 0.0,
  'tick_size': 0.0,
  'lot_size': 0,
  'instrument_type': 'EQ',
  'segment': 'INDICES',
  'exchange': 'NSE'},
 {'instrument_token': 264457,
  'exchange_token': '1033',
  'tradingsymbol': 'NIFTY 200',
  'name': 'NIFTY 200',
  'last_price': 0.0,
  'expiry': '',
  '

In [10]:
inst_df=pd.DataFrame(kite.instruments("NSE"))

In [11]:
def instrumentLookup(instrument_df,symbol):
    """Looks up instrument token for a given script from instrument dump"""
    try:
        return instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]
    except:
        return -1

In [12]:
def fetchOHLC(ticker,interval,duration):
    """extracts historical data and outputs in the form of dataframe"""
    instrument = instrumentLookup(inst_df,ticker)
    data = pd.DataFrame(kite.historical_data(instrument,dt.date.today()-dt.timedelta(duration), dt.date.today(),interval))
    data.set_index("date",inplace=True)
    return data

In [13]:
def fetchOHLCExtended(ticker,inception_date, interval):
    """extracts historical data and outputs in the form of dataframe
       inception date string format - dd-mm-yyyy"""
    instrument = instrumentLookup(inst_df,ticker)
    from_date = dt.datetime.strptime(inception_date, '%d-%m-%Y')
    to_date = dt.date.today()
    data = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])
    temp = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close', 'volume'])

    while True:
        if from_date.date() >= (dt.date.today() - dt.timedelta(100)):
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,dt.date.today(),interval)),ignore_index=True)
            #time.sleep(0.2)
            break
        else:
            to_date = from_date + dt.timedelta(100)
            temp=pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval))
            data=pd.concat([data,temp])
#             data = data.append(pd.DataFrame(kite.historical_data(instrument,from_date,to_date,interval)),ignore_index=True)
            from_date = to_date
            #time.sleep(0.2)
    data.set_index("date",inplace=True)
    return data

In [14]:
def atr(DF,n):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df['H-L']=abs(df['high']-df['low'])
    df['H-PC']=abs(df['high']-df['close'].shift(1))
    df['L-PC']=abs(df['low']-df['close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1,skipna=False)
    df['ATR'] = df['TR'].ewm(com=n,min_periods=n).mean()
    return df['ATR']

In [15]:

def supertrend(DF,n,m):
    """function to calculate Supertrend given historical candle data
        n = n day ATR - usually 7 day ATR is used
        m = multiplier - usually 2 or 3 is used"""
    df = DF.copy()
    df['ATR'] = atr(df,n)
    df["B-U"]=((df['high']+df['low'])/2) + m*df['ATR'] 
    df["B-L"]=((df['high']+df['low'])/2) - m*df['ATR']
    df["U-B"]=df["B-U"]
    df["L-B"]=df["B-L"]
    ind = df.index
    
    for i in range(n,len(df)):
        if df['close'][i-1]<=df['U-B'][i-1]:
            df.loc[ind[i],'U-B']=min(df['B-U'][i],df['U-B'][i-1])
        else:
            df.loc[ind[i],'U-B']=df['B-U'][i]  
            
    for i in range(n,len(df)):
        if df['close'][i-1]>=df['L-B'][i-1]:
            df.loc[ind[i],'L-B']=max(df['B-L'][i],df['L-B'][i-1])
        else:
            df.loc[ind[i],'L-B']=df['B-L'][i]  
            
    df[ 'Strend'+str(n)]=np.nan
    
    for test in range(n,len(df)):
        if df['close'][test-1]<=df['U-B'][test-1] and df['close'][test]>df['U-B'][test]:
            df.loc[ind[test], 'Strend'+str(n)]=df['L-B'][test]
            break
        if df['close'][test-1]>=df['L-B'][test-1] and df['close'][test]<df['L-B'][test]:
            df.loc[ind[test], 'Strend'+str(n)]=df['U-B'][test]
            break
            
    for i in range(test+1,len(df)):
        if df[ 'Strend'+str(n)][i-1]==df['U-B'][i-1] and df['close'][i]<=df['U-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['U-B'][i]
        elif  df[ 'Strend'+str(n)][i-1]==df['U-B'][i-1] and df['close'][i]>=df['U-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['L-B'][i]
        elif df[ 'Strend'+str(n)][i-1]==df['L-B'][i-1] and df['close'][i]>=df['L-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['L-B'][i]
        elif df[ 'Strend'+str(n)][i-1]==df['L-B'][i-1] and df['close'][i]<=df['L-B'][i]:
            df.loc[ind[i], 'Strend'+str(n)]=df['U-B'][i]
    df.drop(['ATR' , 'B-U' , 'B-L' , 'U-B' , 'L-B'], axis=1, inplace=True)
    return df


In [16]:
def EMA(DF, period):
    df=DF.copy()
    df['ma_'+str(period)]=round(df.iloc[:,0].ewm(span=period, adjust=False).mean(),2)
    return df

In [17]:
def MACD(DF,a,b,c):
    df = DF.copy()
    df["MA_Fast"]=df["close"].ewm(span=a,min_periods=a).mean()
    df["MA_Slow"]=df["close"].ewm(span=b,min_periods=b).mean()
    df["MACD"]=df["MA_Fast"]-df["MA_Slow"]
    df["Signal"]=df["MACD"].ewm(span=c,min_periods=c).mean()
    df.dropna(inplace=True)
    return df

In [18]:
def rsi(df, n):
    "function to calculate RSI"
    delta = df["close"].diff().dropna()
    u = delta * 0
    d = u.copy()
    u[delta > 0] = delta[delta > 0]
    d[delta < 0] = -delta[delta < 0]
    u[u.index[n-1]] = np.mean( u[:n]) # first value is average of gains
    u = u.drop(u.index[:(n-1)])
    d[d.index[n-1]] = np.mean( d[:n]) # first value is average of losses
    d = d.drop(d.index[:(n-1)])
    rs = u.ewm(com=n,min_periods=n).mean()/d.ewm(com=n,min_periods=n).mean()
    df['rsi_'+str(n)]=100 - 100 / (1+rs)
    return df

In [25]:
def report(log,withdraw,days,cap):
    rep={}
    rep['Stock']=log['ticker'][0]
    rep['Capital']=cap
    rep['Profit']=round(log['profit'].sum(),1)
    rep['Trades']=len(log.index)/2
    rep['Sum of Profit']=round(log['profit'].loc[log['profit']>0].sum(),1)
    rep['Sum of Loss']=round(log['profit'].loc[log['profit']<0].sum(),1)
    rep['Max Profit']=round(log['profit'].max(),1)
    rep['Max Loss']=round(log['profit'].min(),1)
#     return(pd.DataFrame.from_dict(rep))
    return(rep)

In [26]:
def trade(ticker,capital,position,log,comp_flag):
    df=fetchOHLCExtended(ticker,'27-04-2022',"5minute")
    df=supertrend(df,10,3)
    df=supertrend(df,30,9)

    df=EMA(df,200)
    df.to_csv("Signal.csv")
    profit=0
    withdraw={}
    curr_cap=capital
    
    df["time"]=df.index
    df['time'] =pd.to_datetime(df['time'])
    df['time']=df['time'].dt.time

    t1 = pd.Timestamp('2022-03-16 09:30:00').time()
    t2 = pd.Timestamp('2022-03-16 15:15:00').time()
#     t3 = pd.Timestamp('2022-03-16 15:10:00').time()
    df=df[(df['time']>=t1)]
    df=df[(df['time']<=t2)]
    

    
    for i in range(1,len(df.index)):
#         if (df['time'].dt.time[i]>=t1) and (df['time'].dt.time[i]<=t2):

        if(position==0):
            if ((df['close'][i]>df['Strend10'][i-1]) and (df['close'][i]>df['ma_200'][i]) and (df['close'][i]<df['Strend10'][i-1]*1.005) and (df['close'][i]>df['Strend30'][i-1]) and (df['ma_200'][i]<df['Strend10'][i])):
                quantity=(curr_cap/df['close'][i])//1
                bp=quantity*df['close'][i]
                temp=pd.DataFrame({'ticker':ticker,'action': "Buy",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':0 },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=1
        else:
            if((df['close'][i]<log['price'][-1]*0.995)):
                sp= quantity*df['close'][i]
                profit=sp-bp
                if comp_flag:
                    curr_cap+=profit
                else:
                    curr_cap+=profit
                    if curr_cap>capital:
                        withdraw[str(df.index[i])]=curr_cap-capital
                        curr_cap=capital
                temp=pd.DataFrame({'ticker':ticker,'action': "Stop",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=0
                
            elif((df['close'][i]<df['Strend10'][i-1])):
                sp= quantity*df['close'][i]
                profit=sp-bp
                if comp_flag:
                    curr_cap+=profit
                else:
                    curr_cap+=profit
                    if curr_cap>capital:
                        withdraw[str(df.index[i])]=curr_cap-capital
                        curr_cap=capital
                temp=pd.DataFrame({'ticker':ticker,'action': "Sell",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
                log=pd.concat([log,temp])
                position=0
#             elif((df['time'][i]==t2)):
#                 sp= quantity*df['close'][i]
#                 profit=sp-bp
#                 if comp_flag:
#                     curr_cap+=profit
#                 else:
#                     curr_cap+=profit
#                     if curr_cap>capital:
#                         withdraw[str(df.index[i])]=curr_cap-capital
#                         curr_cap=capital
#                 temp=pd.DataFrame({'ticker':ticker,'action': "Auto",'price': df['close'][i],'quantity':quantity,'current capital':curr_cap,'profit':profit },index=[df.index[i]])
#                 log=pd.concat([log,temp])
#                 position=0

    return (log,withdraw)
#             print("modifying SL to "+ df['ma_20'][-1])        
#     df=EMA(df,200)
#     if(position==0):
#         if ((df['close'][-1]>df['Strend'][-1]) and (df['close'][-1]>df['ma_200'][-1])):
#             print("buying at "+str(df['close'][-1]))
#             print("stoploss at "+ str(df['ma_20'][-1]))
#     else:
#         print("modifying SL to "+ df['ma_20'][-1])
#     print(df.iloc[-1,:])
        

In [81]:
log=pd.DataFrame()
log,withdraw=trade("MCDOWELL-N'",100000,0,log,True)
log

InputException: invalid token

In [53]:

report(log,withdraw,1,100000)

{'Stock': 'BAJAJ-AUTO',
 'Capital': 100000,
 'Profit': -419.9,
 'Trades': 123.0,
 'Sum of Profit': 30967.4,
 'Sum of Loss': -31387.3,
 'Max Profit': 3738.7,
 'Max Loss': -916.8}

<IPython.core.display.Javascript object>

In [28]:
log['action'].value_counts()

Buy     110
Sell     81
Stop     28
Name: action, dtype: int64

In [29]:
log.to_csv("Sample_log.csv")

In [28]:
stocks=['ACC', 'ADANIENT', 'ADANIPORTS', 'ADANIPOWER']

In [84]:
stocks = ['AARTIIND',
 'ABB',
 'ABBOTINDIA',
 'ACC',
 'ADANIENT',
 'ADANIPORTS',
 'ABCAPITAL',
 'ABFRL',
 'ALKEM',
 'AMBUJACEM',
 'APOLLOHOSP',
 'APOLLOTYRE',
 'ASHOKLEY',
 'ASIANPAINT',
 'ASTRAL',
 'ATUL',
 'AUBANK',
 'AUROPHARMA',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BALKRISIND',
 'BALRAMCHIN',
 'BANDHANBNK',
 'BANKBARODA',
 'BATAINDIA',
 'BERGEPAINT',
 'BEL',
 'BHARATFORG',
 'BHEL',
 'BPCL',
 'BHARTIARTL',
 'BIOCON',
 'BSOFT',
 'BOSCHLTD',
 'BRITANNIA',
 'CANFINHOME',
 'CANBK',
 'CHAMBLFERT',
 'CHOLAFIN',
 'CIPLA',
 'CUB',
 'COALINDIA',
 'COFORGE',
 'COLPAL',
 'CONCOR',
 'COROMANDEL',
 'CROMPTON',
 'CUMMINSIND',
 'DABUR',
 'DALBHARAT',
 'DEEPAKNTR',
 'DELTACORP',
 'DIVISLAB',
 'DIXON',
 'DLF',
 'LALPATHLAB',
 'DRREDDY',
 'EICHERMOT',
 'ESCORTS',
 'EXIDEIND',
 'GAIL',
 'GLENMARK',
 'GMRINFRA',
 'GODREJCP',
 'GODREJPROP',
 'GRANULES',
 'GRASIM',
 'GUJGASLTD',
 'GNFC',
 'HAVELLS',
 'HCLTECH',
 'HDFCAMC',
 'HDFCBANK',
 'HDFCLIFE',
 'HEROMOTOCO',
 'HINDALCO',
 'HAL',
 'HINDCOPPER',
 'HINDPETRO',
 'HINDUNILVR',
 'HDFC',
 'ICICIBANK',
 'ICICIGI',
 'ICICIPRULI',
 'IDFCFIRSTB',
 'IDFC',
 'IBULHSGFIN',
 'INDIAMART',
 'IEX',
 'IOC',
 'IRCTC',
 'IGL',
 'INDUSTOWER',
 'INDUSINDBK',
 'NAUKRI',
 'INFY',
 'INTELLECT',
 'INDIGO',
 'IPCALAB',
 'ITC',
 'JINDALSTEL',
 'JKCEMENT',
 'JSWSTEEL',
 'JUBLFOOD',
 'KOTAKBANK',
 'L&TFH',
 'LTTS',
 'LTIM',
 'LT',
 'LAURUSLABS',
 'LICHSGFIN',
 'LUPIN',
 'MGL',
 'M&MFIN',
 'M&M',
 'MANAPPURAM',
 'MARICO',
 'MARUTI',
 'MFSL',
 'METROPOLIS',
 'MOTHERSON',
 'MPHASIS',
 'MRF',
 'MCX',
 'MUTHOOTFIN',
 'NATIONALUM',
 'NAVINFLUOR',
 'NESTLEIND',
 'NMDC',
 'NTPC',
 'OBEROIRLTY',
 'ONGC',
 'OFSS',
 'PAGEIND',
 'PERSISTENT',
 'PETRONET',
 'PIIND',
 'PIDILITIND',
 'PEL',
 'POLYCAB',
 'PFC',
 'POWERGRID',
 'PNB',
 'PVRINOX',
 'RAIN',
 'RBLBANK',
 'RECLTD',
 'RELIANCE',
 'SBICARD',
 'SBILIFE',
 'SHREECEM',
 'SHRIRAMFIN',
 'SIEMENS',
 'SRF',
 'SBIN',
 'SAIL',
 'SUNPHARMA',
 'SUNTV',
 'SYNGENE',
 'TATACHEM',
 'TATACOMM',
 'TCS',
 'TATACONSUM',
 'TATAMOTORS',
 'TATAPOWER',
 'TATASTEEL',
 'TECHM',
 'FEDERALBNK',
 'INDIACEM',
 'INDHOTEL',
 'RAMCOCEM',
 'TITAN',
 'TORNTPHARM',
 'TRENT',
 'TVSMOTOR',
 'ULTRACEMCO',
 'UBL',
 'UPL',
 'VEDL',
 'IDEA',
 'VOLTAS',
 'WHIRLPOOL',
 'WIPRO',
 'ZEEL',
 'ZYDUSLIFE']

In [85]:
import warnings
warnings.filterwarnings('ignore')

start_time = time.time()

comp=pd.DataFrame()
for i in stocks:
    log=pd.DataFrame()
    log,withdraw=trade(i,100000,0,log,True)
    comp = comp.append(report(log,withdraw,1,100000), ignore_index=True)
    
lap=time.time()
time1=lap-start_time
print (time1)



95.45569086074829


In [86]:
comp

,Stock,Capital,Profit,Trades,Sum of Profit,Sum of Loss,Max Profit,Max Loss
0,UPL,100000,25756.0,100.0,64304.3,-38548.3,4444.7,-3181.1
1,VEDL,100000,-10414.1,112.0,47137.8,-57552.0,11398.5,-6130.5
2,IDEA,100000,-7132.0,14.0,543.5,-7675.4,543.5,-1075.1
3,VOLTAS,100000,-202.7,109.0,35604.5,-35807.2,4209.8,-1514.7
4,WHIRLPOOL,100000,-11949.7,103.0,22220.0,-34169.7,3430.4,-1703.0
5,WIPRO,100000,-1237.6,103.0,33313.8,-34551.4,3168.2,-3192.8
6,ZEEL,100000,-4450.0,90.0,32202.1,-36652.0,3645.0,-3185.0
7,ZYDUSLIFE,100000,11407.6,123.0,50118.4,-38710.9,6445.2,-2149.6


In [87]:
comp['Profit'].max()

25756.0

In [88]:
temp4=comp.copy()

In [70]:
temp1

,Stock,Capital,Profit,Trades,Sum of Profit,Sum of Loss,Max Profit,Max Loss
0,AARTIIND,100000,-4052.1,113.0,39512.2,-43564.3,5281.2,-4169.7
1,ABB,100000,15666.3,147.5,67217.7,-51551.4,7530.6,-2859.5
2,ABBOTINDIA,100000,5236.3,124.0,34915.7,-29679.4,2803.0,-1100.8
3,ACC,100000,27176.8,101.0,58081.4,-30904.6,10386.6,-2985.9
4,ADANIENT,100000,90450.2,118.5,144808.3,-54358.1,47824.0,-6604.5
...,...,...,...,...,...,...,...,...
102,JINDALSTEL,100000,-14339.4,128.0,45505.9,-59845.2,5162.6,-14011.8
103,JKCEMENT,100000,7424.9,123.0,50447.3,-43022.4,5664.6,-2129.8
104,JSWSTEEL,100000,-8770.2,123.0,40571.0,-49341.2,4114.5,-10818.5
105,JUBLFOOD,100000,13654.6,106.0,46833.0,-33178.5,3928.5,-1257.5


In [77]:
temp2

,Stock,Capital,Profit,Trades,Sum of Profit,Sum of Loss,Max Profit,Max Loss
0,L&TFH,100000,-10113.3,138.0,37197.3,-47310.6,3080.0,-2441.4
1,LTTS,100000,3237.0,115.0,51974.3,-48737.3,6968.7,-4331.2
2,LTIM,100000,4059.3,129.0,59430.8,-55371.5,6111.2,-2167.5
3,LT,100000,14595.1,126.0,54732.2,-40137.1,4977.6,-2180.8
4,LAURUSLABS,100000,-2976.6,98.0,33281.2,-36257.8,4583.2,-2924.1
5,LICHSGFIN,100000,5416.5,130.0,50233.0,-44816.5,4767.7,-1883.0
6,LUPIN,100000,6051.3,111.0,38844.0,-32792.7,6349.6,-1504.8
7,MGL,100000,33997.3,108.0,69148.6,-35151.3,10449.3,-3880.8
8,M&MFIN,100000,32644.1,128.0,84451.3,-51807.1,10362.6,-1593.0
9,M&M,100000,13733.5,127.5,53937.1,-40203.5,7322.5,-1159.2


In [89]:
result = pd.concat([temp1, temp2, temp3,temp4], axis=0, ignore_index=True)

In [90]:
result

,Stock,Capital,Profit,Trades,Sum of Profit,Sum of Loss,Max Profit,Max Loss
0,AARTIIND,100000,-4052.1,113.0,39512.2,-43564.3,5281.2,-4169.7
1,ABB,100000,15666.3,147.5,67217.7,-51551.4,7530.6,-2859.5
2,ABBOTINDIA,100000,5236.3,124.0,34915.7,-29679.4,2803.0,-1100.8
3,ACC,100000,27176.8,101.0,58081.4,-30904.6,10386.6,-2985.9
4,ADANIENT,100000,90450.2,118.5,144808.3,-54358.1,47824.0,-6604.5
...,...,...,...,...,...,...,...,...
182,VOLTAS,100000,-202.7,109.0,35604.5,-35807.2,4209.8,-1514.7
183,WHIRLPOOL,100000,-11949.7,103.0,22220.0,-34169.7,3430.4,-1703.0
184,WIPRO,100000,-1237.6,103.0,33313.8,-34551.4,3168.2,-3192.8
185,ZEEL,100000,-4450.0,90.0,32202.1,-36652.0,3645.0,-3185.0


In [96]:
df_sorted = result.sort_values('Profit', ascending=False)


In [97]:
df_sorted

,Stock,Capital,Profit,Trades,Sum of Profit,Sum of Loss,Max Profit,Max Loss
4,ADANIENT,100000,90450.2,118.5,144808.3,-54358.1,47824.0,-6604.5
142,PFC,100000,56910.1,119.0,96833.2,-39923.2,13810.5,-2834.7
176,TVSMOTOR,100000,55857.8,120.0,95140.2,-39282.4,5925.1,-2223.9
30,BHEL,100000,54634.5,125.0,110667.4,-56032.9,14632.0,-7608.0
171,INDHOTEL,100000,42693.6,142.0,96043.9,-53350.3,10070.0,-1989.9
...,...,...,...,...,...,...,...,...
47,COROMANDEL,100000,-15923.9,116.0,22954.2,-38878.1,4092.0,-2563.6
33,BIOCON,100000,-16468.4,114.0,25533.6,-42002.0,4178.5,-5362.7
123,MPHASIS,100000,-17192.0,123.0,31531.8,-48723.9,4311.5,-4721.0
127,NATIONALUM,100000,-19720.4,109.0,22978.5,-42698.9,3448.1,-2981.2


In [98]:
df_sorted.to_csv("profit sum.csv")